In [99]:
import pandas as pd
import glob
DATAPATH='../../../paper_data_v3/irat_enet/results/animal_predictions/animal_combined_with_irat.csv'
FILES=glob.glob(DATAPATH)

def starts_with_alphabetical(s):
    if len(s) > 0 and (s[0].isalpha() or s[0]=='_'):
        return True
    else:
        return False

def getInfo(term="A/swine/Minnesota/A02711856/2022",variables=None,lookup=False):
    if not lookup:
        #print(term)
        if '/' in term:
            T= term.split('/')
            if starts_with_alphabetical(T[2]):
                return T[2].replace('_','')
            else:
                return T[1].replace('_','')
        if '_' in term:
            T= term.split('_')
            if starts_with_alphabetical(T[2]):
                return T[2].replace('_','')
            else:
                return T[1].replace('_','')
            
    from Bio import Entrez
    print(term)
    Entrez.email = "ishanu@uchicago.edu"    
    handle = Entrez.esearch(db="nucleotide", term=term)
    record = Entrez.read(handle)
    Ldict={}
    for ID in record["IdList"]:
        print(ID)
        handle = Entrez.efetch(db="nucleotide", id="2289156410", rettype="gb", retmode="json")
        XML=handle.read()
        LINES=[x.strip() for x in XML.split('\n')]
        LINES=[x[1:]+x[0].replace('/','') for x in LINES if '/' in x[:1] and '=' in x]
        Ldict[ID]={x.split('=')[0]:x.split('=')[1] for x in LINES }
        break
    df=pd.DataFrame(Ldict).transpose().drop_duplicates().transpose()   
    if df.empty:
        return term
    return df.loc['country'].values[0]    

def getCountry(row):
    #print(row.id)
    return getInfo(row.id)

In [100]:
seqf=pd.read_csv(FILES[0])
seqf=seqf.rename(columns={'name':'id','emergence':'IRATe','impact_risk':'IRATi'})
seqf['country']=seqf.apply(getCountry, axis=1)
#seqf.sort_values('country').to_csv('seqf3.csv')

rnames={'OHiggins':'Chile Region del Libertador General Bernardo OHiggins', 'AK': 'Alaska',
        'MB':'Canada Manitoba','gull':'Latvia','NL': 'Canada NL','AB': 'Canada: AB','BC':'Canada: BC'}

import numpy as np
country=[]
for x in seqf.country.values:
    if x in rnames.keys():
        x=rnames[x]
    country=np.append(country,x)
seqf.country=country

pd.DataFrame(list(set(seqf.country.values)),columns=['country']).to_csv('addressstrings.csv')

In [101]:
#! ./geocodelist.py -i addressstrings.csv -o geores -A country

In [102]:
seqfa[~seqfa.latitude.isna()]#.index.values

,id,subtype,date,ha_accession,ha_sequence,na_accession,na_sequence,ha_risk,na_risk,geom_mean_risk,...,mean_high_acceptable_impact,formatted_address,latitude,longitude,accuracy,google_place_id,type,postcode,number_of_results,status
country,,,,,,,,,,,,,,,,,,,,,
Aichi,A/chicken/Aichi/22A3T/2022,H5N1,2022-12-05,EPI2747722,MENIVLLLATVSLVKSDQICIGYHANNSTEQVDTIMEKNVTVTHAQ...,EPI2747721,MNPNQRIITTGSICMVIGIVSLMLQIGNIISIWVSHSIQTGNQHQP...,0.005260,0.000405,0.001459,...,NaN,"Aichi, Japan",35.018251,137.292389,APPROXIMATE,ChIJ5zKTSdW6BGARp0EVGF2mX-s,"administrative_area_level_1,political",NaN,1.0,OK
Alaska,A/northern_pintail/Alaska/USGS-0335/2022,H5N1,2022-09-14,EPI2909469,MENIVLLLAIISLVKSDQICIGYHANNSTEQVDTIMEKNVTVTHAQ...,EPI2909467,MNPNQRIITTGSICMVIGIVSLMLQIGNIISIWVSHSIQTGNQYQP...,0.005253,0.000153,0.000898,...,NaN,"Alaska, USA",63.588753,-154.493062,APPROXIMATE,ChIJG8CuwJzfAFQRNduKqSde27w,"administrative_area_level_1,political",NaN,1.0,OK
Alaska,A/red_fox/Alaska/22-016300-001-original/2022,H5N1,2022-05-10,EPI2473508,MENIVLLLAIISLVESDQICIGYHANNSTEQVDTIMEKNVTVTHAQ...,EPI2473505,MNPNQRIITTGSICMVIGIVSLMLQIGNIISIWVSHSIQTGNQYQP...,0.005260,0.000153,0.000898,...,NaN,"Alaska, USA",63.588753,-154.493062,APPROXIMATE,ChIJG8CuwJzfAFQRNduKqSde27w,"administrative_area_level_1,political",NaN,1.0,OK
Alaska,A/glaucous_gull/Alaska/23-025003-001-original/...,H5N1,2023-07-22,EPI2895279,MENIVLLLAIISLVKSDQICVGYHANNSTEQVDTIMEKNVTVTHAQ...,EPI2895278,MNPNQRIITTGSICMVIGIVSLMLQIGNIISIWVSHSIQTGNQYQP...,0.005269,0.000227,0.001094,...,NaN,"Alaska, USA",63.588753,-154.493062,APPROXIMATE,ChIJG8CuwJzfAFQRNduKqSde27w,"administrative_area_level_1,political",NaN,1.0,OK
Alaska,A/manx_shearwater/Alaska/23-025002-003-origina...,H5N1,2023-07-19,EPI2895287,MENIVLLLAIISLVKSDQICIGYHANNSTEQVDTIMEKNVTVTHAQ...,EPI2895286,MNPNQRIITTGSICMVIGIVSLMLQIGNIISIWVSHSIQTGNQYQP...,0.005280,0.000227,0.001095,...,NaN,"Alaska, USA",63.588753,-154.493062,APPROXIMATE,ChIJG8CuwJzfAFQRNduKqSde27w,"administrative_area_level_1,political",NaN,1.0,OK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zhejiang,A/chicken/Zhejiang/S1102/2020(H10N3),H10N3,2020-05-07,EPI2213793,MYKIVVIIAFLGTVKALDKICLGHHAVANGTIVKTLTNEQEEVTNA...,EPI2213792,MNPNQKIITLGVVNTTLSTLALLIGIGNQIFNTVIHEKVGVHQTVV...,0.078976,0.005092,0.020054,...,NaN,"Zhejiang, China",30.265550,120.153600,APPROXIMATE,ChIJv8wKpTQSSTQRt3wYNsVx74E,"administrative_area_level_1,political",NaN,1.0,OK
Zhejiang,A/chicken/Zhejiang/S1172/2021(H10N3),H10N3,2021-05-08,EPI2213801,MYKTVVIIAFLGTVKALDKICLGHHAVANGTIVKTLTNEQEEVTNA...,EPI2213800,MNPNQKIITLGVVNTTLSTLALLIGIGNQIFNIIIHEKVGVHQTVV...,0.078976,0.005092,0.020054,...,NaN,"Zhejiang, China",30.265550,120.153600,APPROXIMATE,ChIJv8wKpTQSSTQRt3wYNsVx74E,"administrative_area_level_1,political",NaN,1.0,OK
Zhejiang,A/chicken/Zhejiang/ZJ79/2021,H10N3,2021-04-03,EPI1884758,MYKIVVIIAFLGTVKALDKICLGHHAVANGTIVKTLTNEQEEVTNA...,EPI1884760,MNPNQKIITLGVVNTTLSTLALLIGIGNQIFNIVIHEKVGVHQTVV...,0.079099,0.005092,0.020070,...,NaN,"Zhejiang, China",30.265550,120.153600,APPROXIMATE,ChIJv8wKpTQSSTQRt3wYNsVx74E,"administrative_area_level_1,political",NaN,1.0,OK


In [103]:
dff=pd.read_csv('geores',index_col=0)
dff=dff.set_index('input_string')
#dff.index=list(set(seqf.country.values))
dff.index.name='country'
seqfa=seqf.set_index('country').join(dff)
seqfb=seqfa[~seqfa.latitude.isna()]
seqfb.to_csv('finalseq.csv')

In [104]:
seqfa[['latitude',
       'longitude']]

,latitude,longitude
country,,
Aichi,35.018251,137.292389
Akita,NaN,NaN
Alaska,63.588753,-154.493062
Alaska,63.588753,-154.493062
Alaska,63.588753,-154.493062
...,...,...
belgium,50.503887,4.469936
release,NaN,NaN
release,NaN,NaN


In [105]:
seqfa[seqfa.subtype=='H7N9'][['id','emergence_score','IRATe']]

,id,emergence_score,IRATe
country,,,
Hebei,A/Chicken/Hebei/1009/2020,NaN,3.349192
Hebei,A/Chicken/Hebei/1011/2021,NaN,3.346242
Hong Kong,A/Hong Kong/125/2017,6.5,7.732746
InnerMongolia,A/Camel/Inner_Mongolia/XL/2020,NaN,3.466755
Korea,A/wild_duck/Korea/H296/2020,NaN,3.460336
NorthernChina,A/chicken/Northern_China/SX1616/2020(H7N9),NaN,3.439851
NorthernChina,A/chicken/Northern_China/BJ3426/2020(H7N9),NaN,3.396696
Shanghai,A/Shanghai/02/2013,6.4,7.732746
Shanxi,A/Chicken/Shanxi/1012/2021,NaN,3.349152


In [97]:
seqf[seqf.subtype=='H7N9'][['id','emergence_score','IRATe']]

,id,emergence_score,IRATe
0,A/Hong Kong/125/2017,6.5,7.732746
1,A/Shanghai/02/2013,6.4,7.732746
18,A/chicken/Tennessee/17-007431-3/2017,3.1,3.354696
20,A/chicken/Tennessee/17-007147-2/2017,2.8,2.988541
6186,A/Camel/Inner_Mongolia/XL/2020,NaN,3.466755
6190,A/wild_duck/Korea/H296/2020,NaN,3.460336
6191,A/mallard/South_Korea/20X-20/2021,NaN,3.458805
6196,A/chicken/Northern_China/SX1616/2020(H7N9),NaN,3.439851
6217,A/Chicken/Yunnan/1008/2021,NaN,3.401992
6222,A/chicken/Northern_China/BJ3426/2020(H7N9),NaN,3.396696


In [96]:
seqf.columns

Index(['id', 'subtype', 'date', 'ha_accession', 'ha_sequence', 'na_accession',
       'na_sequence', 'ha_risk', 'na_risk', 'geom_mean_risk', 'IRATe',
       'emergence_low', 'emergence_high', 'is_irat', 'emergence_score',
       'impact_score', 'mean_low_acceptable_emergence',
       'mean_high_acceptable_emergence', 'mean_low_acceptable_impact',
       'mean_high_acceptable_impact', 'country'],
      dtype='object')

In [106]:
seqfa[seqfa.id=='A/Shanghai/02/2013']

,id,subtype,date,ha_accession,ha_sequence,na_accession,na_sequence,ha_risk,na_risk,geom_mean_risk,...,mean_high_acceptable_impact,formatted_address,latitude,longitude,accuracy,google_place_id,type,postcode,number_of_results,status
country,,,,,,,,,,,,,,,,,,,,,
Shanghai,A/Shanghai/02/2013,H7N9,2016-04-01,EPI448936,MNTQILVFALIAIIPTNADKICLGHHAVSNGTKVNTLTERGVEVVN...,EPI448938,MNPNQKILCTSATAIIIGAIAVLIGMANLGLNIGLHLKPGCNCSHS...,0.00001,0.00001,0.00001,...,8.32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [108]:
! cat addressstrings.csv | grep Shan

67,Shandong
84,Shanghai
167,Shanxi


In [109]:
! cat geores | grep Shan

142,"Shandong, China",36.6682599,117.02076,APPROXIMATE,ChIJf0-rNUfNjzURZLPhcCMuOtk,"administrative_area_level_1,political",,Shandong,1,OK
181,"Shanxi, China",37.8734299,112.56272,APPROXIMATE,ChIJAZbireuoCjYRL-LvlMM8zpw,"administrative_area_level_1,political",,Shanxi,1,OK
